In [1]:
import pickle

import torch
import numpy as np

from callformer.transformer import CallFormer, ModelDimensions
from callformer.tokenizer import Tokenizer
from callformer.decoding import BeamSearchDecoder, DecodingOptions

DATA_PATH = "full_samples.pkl"

with open(DATA_PATH, "rb") as f:
    full_samples = pickle.load(f)

STATE_SIZE = len(full_samples[0][-1])

tokenizer = Tokenizer()

model_dims = ModelDimensions(
                n_vocab=tokenizer.vocab_size,
                n_ctx=100,
                n_state=STATE_SIZE,
                n_head=8,
                n_layer=2,
                n_prefix=1,)


In [2]:
model = CallFormer(model_dims)

embed = torch.from_numpy(np.array(full_samples[0][-1])).unsqueeze(0)

options = DecodingOptions(
    decoder = BeamSearchDecoder,
    decoder_options=dict(beam_size=1,
                         eot=tokenizer.eot,
                         )
)

model.decode(embed, options=options)

torch.Size([1, 28])


AssertionError: 

In [20]:
idx = (3,6,19)
a = torch.empty(2, 28).fill_(-np.inf)
a[:, idx] = 0
b = torch.rand_like(a)
c = torch.nn.functional.log_softmax(b+a)
for i in range(c.shape[0]):
    for logprob, j in zip(*c[i].topk(4, dim=-1)):
        print (logprob.item(), j.item())

-0.9177658557891846 6
-0.9518564939498901 19
-1.5391385555267334 3
-inf 18
-0.8997703194618225 19
-1.0324859619140625 3
-1.4387824535369873 6
-inf 21


/tmp/ipykernel_3841639/1602605086.py:5: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  c = torch.nn.functional.log_softmax(b+a)


In [24]:
import numpy as np
from callformer.commands import CommandsList

cmdl1 = CommandsList()
for i, tokens in enumerate(cmdl1.valid_tokens()):
    idx = np.random.randint(0, len(tokens))
    cmdl1.add_token(tokens[idx])
    if i == 2:
        break

cmdl2 = CommandsList()
for token in cmdl1.sequence:
    cmdl2.add_token(token)

print (cmdl1.tokenizer.decode(cmdl1.sequence))

[4, 11, 15]
